![Funcional Health Tech](logo.png "Logo")

**COVID19 - Análise do Surto Brasileiro**

# Pré-Processamento de Dados

Neste *notebook* integramos e transformamos diversas fontes de dados, para que estejam disponíveis em análises posteriores. Empregaremos aqui as seguintes fontes:

  - [Our World in Data](https://ourworldindata.org/coronavirus-source-data)
  - [Banco Mundial](https://datacatalog.worldbank.org/dataset/world-development-indicators)
  

## Preliminares
Bibliotecas, configurações e parâmetros.

In [42]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]
import seaborn as sns
sns.set()

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [43]:
plt.style.use('fivethirtyeight')

In [44]:
#os.getcwd()
#os.chdir('covid19-analytics/notebooks')
#os.getcwd()

## Leitura das Fontes Originais

Vamos empregar dados do [Our World in Data](https://ourworldindata.org/coronavirus-source-data ) como nossa fonte principal de dados de COVID-19, mas também a enriqueceremos com outras informações.

### Dados de COVID-19

Dados de infecções e óbitos, além de algumas informações demográficas.

In [45]:
df_epidemy_data = pd.read_csv('../data/ourworldindata.org/coronavirus-source-data/full_data.csv', parse_dates=['date'])
df_epidemy_data.head(2)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0


In [46]:
df_locations_data = pd.read_csv('../data/ourworldindata.org/coronavirus-source-data/locations.csv')
df_locations_data.head(2)

,countriesAndTerritories,location,continent,population_year,population
0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0
1,Albania,Albania,Europe,2020.0,2877800.0


Vamos enriquecer os dados com informações adicionais sobre as localidades.

In [47]:
df_epidemy_data_enriched = df_epidemy_data.merge(df_locations_data, left_on='location', right_on='location', suffixes=('', '_demography'))

df_epidemy_data_enriched['total_cases_per_capita'] = df_epidemy_data_enriched['total_cases'] / df_epidemy_data_enriched['population']
df_epidemy_data_enriched['total_deaths_per_capita'] = df_epidemy_data_enriched['total_deaths'] / df_epidemy_data_enriched['population']

df_epidemy_data_enriched.head(2)

,date,location,new_cases,new_deaths,total_cases,total_deaths,countriesAndTerritories,continent,population_year,population,total_cases_per_capita,total_deaths_per_capita
0,2019-12-31,Afghanistan,0,0,0,0,Afghanistan,Asia,2020.0,38928341.0,0.0,0.0
1,2020-01-01,Afghanistan,0,0,0,0,Afghanistan,Asia,2020.0,38928341.0,0.0,0.0


Faixa de datas para a qual temos dados.

In [48]:
print(f"From {df_epidemy_data_enriched['date'].iloc[0].date()} to {df_epidemy_data_enriched['date'].iloc[-1].date()}")

From 2019-12-31 to 2020-05-03


Salvemos para uso posterior.

In [49]:
df_epidemy_data.to_csv('../data/preprocessed/covid19_progression.csv', index=False)

### Banco Mundial: World Development Indicators

Incorporemos agora dados do Banco Mundial.

In [50]:
df_wdi = pd.read_csv('../data/large/datacatalog.worldbank.org/dataset/world-development-indicators/WDIData.csv')
df_wdi.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.428272,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN


Vamos computar o valor mais recente disponível para os diversos indicadores.

In [51]:
def latest_not_nan(values):
    values = np.array(values)
    values = values[~np.isnan(values)]
    if len(values) > 0:
        return values[-1]
    else:
        return None
    
df_wdi['latest_value_available'] = df_wdi.apply(lambda row: latest_not_nan([row[str(y)] for y in range(2005, 2020)]), axis=1)
df_wdi.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64,latest_value_available
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN,84.510171
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN,90.273687


Quantos indicadores totais temos?

In [52]:
df_wdi['Indicator Name'].nunique()

1429

Escolhamos então alguns indicadores, pois temos centenas deles. Como sugestão, podemos tomar o que o próprio Banco Mundial destaca para os países, por exemplo para o [Brasil](https://data.worldbank.org/country/brazil).

In [53]:
selected_world_bank_indicator_codes = ['NY.GDP.MKTP.CD', # GDP
                                       'SP.POP.TOTL', # Population
                                       'SE.PRM.ENRR', # School enrollment, primary (% gross)
                                       'SE.PRM.TENR',
                                       'EN.ATM.CO2E.PC', # CO2
                                       'SP.DYN.LE00.IN', # life expectancy
                                       'NY.GNP.PCAP.CD', # GNI per capita
                                       ]

In [54]:
df_wdi = df_wdi[df_wdi['Indicator Code'].isin(selected_world_bank_indicator_codes)]

Como fica para um país específico?

In [55]:
df_wdi[df_wdi['Country Name'] == 'Brazil']

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64,latest_value_available
104334,Brazil,BRA,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,...,9.854993e+01,9.732389e+01,9.743620e+01,9.685231e+01,9.720202e+01,9.755117e+01,NaN,NaN,NaN,9.755117e+01
104510,Brazil,BRA,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,6.498859e-01,6.621305e-01,7.017755e-01,7.060494e-01,6.998843e-01,6.764552e-01,...,2.358548e+00,2.505409e+00,2.612934e+00,NaN,NaN,NaN,NaN,NaN,NaN,2.612934e+00
104784,Brazil,BRA,GDP (current US$),NY.GDP.MKTP.CD,1.516557e+10,1.523685e+10,1.992629e+10,2.302148e+10,2.121189e+10,2.179004e+10,...,2.465189e+12,2.472806e+12,2.455994e+12,1.802214e+12,1.796275e+12,2.053595e+12,1.868626e+12,NaN,NaN,1.868626e+12
104818,Brazil,BRA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.236000e+04,1.281000e+04,1.210000e+04,1.016000e+04,8.930000e+03,8.670000e+03,9.140000e+03,NaN,NaN,9.140000e+03
105018,Brazil,BRA,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,5.414300e+01,5.463400e+01,5.513000e+01,5.562700e+01,5.612100e+01,5.661000e+01,...,7.420900e+01,7.448300e+01,7.474500e+01,7.499400e+01,7.523000e+01,7.545600e+01,7.567200e+01,NaN,NaN,7.567200e+01
105372,Brazil,BRA,"Population, total",SP.POP.TOTL,7.217923e+07,7.431134e+07,7.651433e+07,7.877266e+07,8.106457e+07,8.337353e+07,...,1.992873e+08,2.010359e+08,2.027637e+08,2.044718e+08,2.061631e+08,2.078338e+08,2.094693e+08,NaN,NaN,2.094693e+08
105505,Brazil,BRA,"School enrollment, primary (% gross)",SE.PRM.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,...,1.345204e+02,1.139198e+02,1.142547e+02,1.127733e+02,1.139454e+02,1.154478e+02,NaN,NaN,NaN,1.154478e+02


Vamos deixar apenas algumas colunas.

In [56]:
df_wdi = df_wdi[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'latest_value_available']]
df_wdi.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,latest_value_available
17,Arab World,ARB,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,85.384220
193,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,4.886988


In [57]:
df_wdi[df_wdi['Country Name'] == 'Brazil'].head()

,Country Name,Country Code,Indicator Name,Indicator Code,latest_value_available
104334,Brazil,BRA,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,9.755117e+01
104510,Brazil,BRA,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2.612934e+00
104784,Brazil,BRA,GDP (current US$),NY.GDP.MKTP.CD,1.868626e+12
104818,Brazil,BRA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,9.140000e+03
105018,Brazil,BRA,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,7.567200e+01


In [58]:
df_wdi_pivot = df_wdi.pivot(index='Country Name', columns='Indicator Code', values='latest_value_available')
df_wdi_pivot.head()

Indicator Code,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GNP.PCAP.CD,SE.PRM.ENRR,SE.PRM.TENR,SP.DYN.LE00.IN,SP.POP.TOTL
Country Name,,,,,,,
Afghanistan,0.293946,1.936297e+10,550.0,103.99616,NaN,64.486,37172386.0
Albania,1.978763,1.510250e+10,4860.0,106.99342,97.44221,78.458,2866376.0
Algeria,3.735520,1.737580e+11,3920.0,109.88282,99.64750,76.693,42228429.0
American Samoa,NaN,6.360000e+08,NaN,NaN,NaN,NaN,55465.0
Andorra,5.832906,3.236544e+09,NaN,NaN,NaN,NaN,77006.0


Salvemos os indicadores para cada país.

In [59]:
df_wdi_pivot.to_csv('../data/preprocessed/world_bank_selected_wdi.csv')

In [60]:
df_epidemy_data_enriched = df_epidemy_data_enriched.merge(df_wdi_pivot, left_on='location', right_on='Country Name')
#df_epidemy_data_enriched.head()

Salvemos também a versão dos dados epidêmicos enriquecida com os indicadores do Banco Mundial.

In [61]:
df_epidemy_data_enriched.to_csv('../data/preprocessed/covid19_wdi-enriched_progression.csv', index=False)

## Estruturação para Estudos

Convém agora colocar os dados escolhidos em formatos mais adequados para as análises seguintes.


### Tabelas Completas

Primeiramente, separemos os casos detectados dos óbitos, tendo os países como colunas. Isso facilitará a comparação temporal absoluta dos diversos países.

In [62]:
df_total_cases = df_epidemy_data.pivot(index='date', columns='location', values='total_cases')
#df_total_cases.tail()

In [63]:
df_total_cases.to_csv('../data/preprocessed/covid19_total_cases.csv')

In [64]:
df_total_deaths = df_epidemy_data.pivot(index='date', columns='location', values='total_deaths')
#df_total_deaths.tail()

In [65]:
df_total_deaths.to_csv('../data/preprocessed/covid19_total_deaths.csv')

Precisamos também ajustar os dados para mostrarem o progresso a partir do primeiro caso detectado, de modo que a progressão relativa da epitemia também possa ser analisada. Para tanto, a seguinte função nos dá, para cada métrica escolhida (`'total_cases'`, `'total_cases_per_capita'`, `'total_cases'` e `'total_cases_per_capita'`) tanto o *dataframe* apropriado quanto um dicionário que mapeia, para cada país, quando foi o primeiro momento em que e métrica escolhida foi diferente de zero.

In [66]:
def build_dataframe_with_metric_per_epidemy_day(metric_name, base_metric_name='total_cases'):
    first_dates_per_location = {}
    progresses_per_epidemy_day = []
    
    locations = df_epidemy_data_enriched['location'].unique()
    
    for location in locations:
        
        # the first date is calculated from a common base metric, so that multiple metrics can be properly compared.
        first_date = df_epidemy_data_enriched[(df_epidemy_data_enriched['location'] == location) & \
                                              (df_epidemy_data_enriched[base_metric_name] != 0)]['date'].iloc[0]
        
        df_location = df_epidemy_data_enriched[df_epidemy_data_enriched['location'] == location]
        df_location = df_location[df_location['date'] >= first_date]
        s_selected_metric = df_location[metric_name].rename(location).reset_index(drop=True)
        progresses_per_epidemy_day.append(s_selected_metric)

        first_dates_per_location[location] = first_date
        
    df_metric_per_epidemy_day = pd.concat(progresses_per_epidemy_day, axis=1)
    return df_metric_per_epidemy_day, first_dates_per_location


In [67]:
df_total_cases_per_epidemy_day, first_cases_per_location = build_dataframe_with_metric_per_epidemy_day(metric_name='total_cases')
#df_total_cases_per_epidemy_day.head()

In [68]:
df_total_cases_per_epidemy_day.to_csv('../data/preprocessed/covid19_total_cases_per_epidemy_day.csv')

In [69]:
df_total_cases_per_capita_per_epidemy_day, first_cases_per_location = \
                            build_dataframe_with_metric_per_epidemy_day(metric_name='total_cases_per_capita')

#df_total_cases_per_capita_per_epidemy_day.head()

In [70]:
df_total_cases_per_capita_per_epidemy_day.to_csv('../data/preprocessed/covid19_total_cases_per_capita_per_epidemy_day.csv')

In [71]:
df_total_deaths_per_epidemy_day, first_deaths_per_location = \
                                    build_dataframe_with_metric_per_epidemy_day(metric_name='total_deaths')

#df_total_deaths_per_epidemy_day.head()

In [72]:
df_total_deaths_per_epidemy_day.to_csv('../data/preprocessed/covid19_total_deaths_per_epidemy_day.csv')

In [73]:
df_total_deaths_per_capita_per_epidemy_day, first_deaths_per_location = \
                                    build_dataframe_with_metric_per_epidemy_day(metric_name='total_deaths_per_capita')

#df_total_deaths_per_capita_per_epidemy_day.head()

In [74]:
df_total_deaths_per_capita_per_epidemy_day.to_csv('../data/preprocessed/covid19_total_deaths_per_capita_per_epidemy_day.csv')

### Para Treino Temporal

Alguns algoritmos empregues valem-se de treinamento que requer particionamento dos dados em janelas de tempo. Para tanto, aqui já realizamos esse pré-processamento, de modo que os dados já estejam no formato adequado para esse tipo de treino.

In [75]:
min_total_cases = 50
min_total_deaths = 1

In [76]:
window_size = 10

In [77]:
locations = df_epidemy_data_enriched['location'].unique()
#print(locations)
len(locations)

174

In [78]:
columns = ['location', 'population', 'total_cases'] + ['relative_day'] + [f't{w}' for w in range(0, window_size)] + ['target']
columns

['location',
 'population',
 'total_cases',
 'relative_day',
 't0',
 't1',
 't2',
 't3',
 't4',
 't5',
 't6',
 't7',
 't8',
 't9',
 'target']

In [79]:
new_rows = []
for location in locations:
    #print(location)
    df_tmp = df_epidemy_data_enriched[df_epidemy_data_enriched['location'] == location]
    
    # minimum number of cases and deaths
    df_tmp = df_tmp[df_tmp['total_cases'] >= min_total_cases]
    df_tmp = df_tmp[df_tmp['total_deaths'] >= min_total_deaths]
    df_tmp = df_tmp.reset_index()
    
    # sliding window
    for i in range(0, len(df_tmp) - window_size):
        df_window = df_tmp.iloc[i:i + window_size]
        df_window_last_pos = df_tmp.iloc[i + window_size - 1]
        df_target = df_tmp.iloc[i + window_size]
        
        features =  [location, df_window_last_pos['population'], df_window_last_pos['total_cases']] + [i + window_size - 1] + list(df_window['new_cases'].values)
        target = df_target['new_cases']
        
        new_row_values = list(features) + [target]
        new_rows.append(new_row_values)
        
        
        #print(features)
        #print(target)
        #print(new_row_values)
        #print(df_window)
        #print(df_target)

df_epidemy_data_with_sliding_windows = pd.DataFrame(new_rows, columns= columns)
df_epidemy_data_with_sliding_windows.head()

,location,population,total_cases,relative_day,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,target
0,Afghanistan,38928341.0,235,9,33,0,16,15,8,27,25,26,43,0,35
1,Afghanistan,38928341.0,270,10,0,16,15,8,27,25,26,43,0,35,29
2,Afghanistan,38928341.0,299,11,16,15,8,27,25,26,43,0,35,29,38
3,Afghanistan,38928341.0,337,12,15,8,27,25,26,43,0,35,29,38,30
4,Afghanistan,38928341.0,367,13,8,27,25,26,43,0,35,29,38,30,56


In [80]:
len(df_epidemy_data_with_sliding_windows)

3958

In [81]:
df_epidemy_data_with_sliding_windows['location'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin',
       'Bermuda', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Cameroon', 'Canada', 'Cayman Islands',
       'Chile', 'China', 'Colombia', 'Costa Rica', "Cote d'Ivoire",
       'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'El Salvador',
       'Estonia', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Guam', 'Guatemala', 'Guinea',
       'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo',
       'Kuwait', 'Latvia', 'Lebanon', 'Liberia', 'Libya', 'Liechtenstein',
       'Lithuania', 'Luxem

In [82]:
df_epidemy_data_with_sliding_windows.to_csv('../data/preprocessed/covid19_sliding_window.csv')